# Data Structures

Common data structures like `linear containers (like array and vector)`, `stack`, `queue`, `deuque (double ended queue)`, `set/map (balanced binary search tree)`, `unordered map/set (hash tables)`, `priority queue` are already implemented in STL library of C++. Hence, implementation of those data structures are ignored since they can be used directly from the standard library. The corresponding C++ reference websites of those data structures are given in the table below:

| Data Structure              | STL Implementation |
| :-------------------------: | :----------------: |
| Fixed-size array            | [std::array](https://en.cppreference.com/w/cpp/container/array)   |
| Dynamic array               | [std::vector](https://en.cppreference.com/w/cpp/container/vector) |
| Stack                       | [std::stack](https://en.cppreference.com/w/cpp/container/stack)   |
| Qeueu                       | [std::queue](https://en.cppreference.com/w/cpp/container/queue)   |
| Deque                       | [std::deque](https://en.cppreference.com/w/cpp/container/deque)   |
| Balanced Binary Search Tree | [std::set](https://en.cppreference.com/w/cpp/container/set), [std::multiset](https://en.cppreference.com/w/cpp/container/multiset), [std::map](https://en.cppreference.com/w/cpp/container/map), [std::multimap](https://en.cppreference.com/w/cpp/container/multimap) |
| Hash Table                  | [std::unordered_set](https://en.cppreference.com/w/cpp/container/unordered_set), [std::unordered_multiset](https://en.cppreference.com/w/cpp/container/unordered_multiset) , [std::unordered_map](https://en.cppreference.com/w/cpp/container/unordered_map), [std::unordered_multimap](https://en.cppreference.com/w/cpp/container/unordered_multimap)  |
| Priority Queue              | [std::priority_queue](https://en.cppreference.com/w/cpp/container/priority_queue) |

* In addition to the data structures above, you can pack together the different type of objects in [`std::tuple`](https://en.cppreference.com/w/cpp/utility/tuple).

### Fenwick (Binary Indexed) Tree

A Fenwick tree is an implicit tree where nodes are consecutively numbered, and parent-child relationships are determined by arithmetic on the node indexes.

An important function in this index arithmetic is the least significant set bit, also called the find first set operation. This is the greatest power of two which divides an index $i$. This is the power of two (1, 2, 4, 8, ...) and not the exponent (0, 1, 2, 3, ...). It can be efficiently computed in two's complement arithmetic as:

$$ lsb(i) = i \& (-i) $$

where $\&$ is the bitwise and operator. A Fenwick tree is most easily understood using a one-based array $A[n]$ with $n$ values. Using half-open interval syntax, let 

$$ A(i,j] = \{A[k]\}_{k=i+1}^{j} $$

the range from $i$ (exclusive) to $j$ (inclusive). The corresponding Fenwich array $F[n]$ stores the range sums

$$F[i] = \Sigma A(i - lsb(i), i]$$

A fictitious node 0 is used in some descriptions, but is never actually accessed and need not be explicitly stored. $lsb(0) = \infty$, but the value is never actually needed. $F[0]$ is considered to contain the sum of empty range $A(0, 0] = \{\}$ with value 0.

A _"Fenwick tree"_ is actually three implicit trees over the same array: the interrogation tree used for translating indexes to prefix sums, the update tree used for updating elements, and the search tree for translating prefix sums to indexes (rank queries). The first two are normally walked upwards, while the third is usually walked downwards.

__Interrogation Tree:__ The interrogation tree is defined so that the parent of node $i$ is $ i - lsb(i) $ where $lsb(i)$ is defined above. Implicit node 0 is the root. Node $i$ has $log_2(lsb(i))$ children $(i+1, i+2, i+4, ..., i+lsb(i)/2)$, and $lsb(i)$ total descendants.

![Fenwick Array Interrogation Tree](./images/FenwickInterrogationTree.png)

__Update Tree:__ The update tree is the mirror image of the interrogation tree. The parent of node $i$ is $i + lsb(i) = (i|i-1)+1$ where $|$ denotes bitwise or operation. This conceptual tree is infinite, but only the part with indexes up to 
$n$ is stored or used. Excluding the fictitious nodes with indexes greater than $n$ it will be a forest of disjoint trees, one for each bit set in the binary representation of $n$. To modify one of the values $A[i]$ add the change to $F[i]$, then $i$'s parent, then its grandparent, and so on, until the index exceeds $n$.

__Search Tree:__ Unlike the other two trees, the search tree is a binary tree, arranged in an order sideways heap. Each node is assigned a height equal to the number of trailing zeros in the binary representation of its index, with the parent and children being the numerically closest index(es) of the adjacent height. Nodes with odd indexes $(lsb(i) = 1)$ are leaves. Nodes with even indexes have the closest two nodes of the next-lowest index as children, $ i \pm lsb(i)/2 $. Node $i$'s parent in the same search tree is

$$(i - lsb(i))|(2lsb(i))$$

Although this tree is potentially infinite, we may define its root to be the highest existing node. The search tree may be considered a combination of the previous two trees. A node's left subtree contains all of its descendants in the update tree, while its right subtree contains all of its descendants in the interrogation tree. A node's parent in the search tree is either its interrogation or update parent (depending on whether the node is a right or left child, respectively), and the other type of parent may be found by multiple upward steps in the search tree.

However, upward traversals in the search tree are uncommon; its primary use is to perform rank queries: given a prefix sum, at what index does it appear? This is done by a downward traversal through the search tree. During the traversal, three variables are maintained: The current node's index, the rank being sought in the subtree rooted at the current node, and a "fallback index" to be returned if the rank sought is greater than can be found in the subtree. Each step, either the current node is a fictitious node (index greater than $n$), or we must decide if the position sought is to the left or right of the end of the current node. If the rank sought is less than the Fenwick array value $F[i]$ for the current node, we must search its left subtree. If it is greater, search its right subtree. If it is equal, the direction chosen depends on how you wish to handle searches for sums lying exactly between two nodes.

In [1]:
#include <bits/stdc++.h>

// implementation of Fenwick Tree
// TODO: implement search algorithm for fenwick tree
class FenwickTree
{
        std::vector<int> tree_;

    public:
        FenwickTree(const std::vector<int>& arr): tree_(arr.size(), 0)
        {
            for (size_t i=0; i<arr.size(); i++)
                update(i, arr[i]);   
        }

        void update(size_t index, int delta_on_index)
        {
            index++;
            while(index < tree_.size())
            {
                tree_[index] += delta_on_index;
                index += index & (-index);
            }
        }

        int sum(size_t index)
        {
            int result = 0;
            index++;
            while(index > 0)
            {
                result += tree_[index];
                index -= index & (-index);
            }
            return result;
        }

        int sum(size_t start, size_t end)
        {
            int result = sum(end);
            result -= sum(--start);
            return result;
        }

        const std::vector<int>& getTree() { return tree_; }
};

In [2]:
// creating a fenwick tree

std::vector<int> arr = {2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9};
FenwickTree tree(arr);
tree.getTree()

{ 0, 2, 3, 1, 7, 2, 5, 4, 21, 6, 13, 8 }

In [3]:
// sum range query starting from zero to given index
tree.sum(4) // sum of first 5 elements = 9

9

In [4]:
// sum range query between two given index (inclusive)
tree.sum(7, 10) // sum of the elements at index 7 8 9 10 = 26

26

In [5]:
// sum range query after modifying one of the elements
tree.update(7, 42 - arr[7] /* delta on the index */); // changing the element at index 7 with 42
arr[7] = 42;
tree.sum(7, 10) // after the element update the sum should be 63

63